# Imports

In [ ]:
import numpy as np
import numba
import random
import networkx as nx
from dill import dump, load
import os

# Function Definitions

## Get DiGraph Origin

In [ ]:
def get_origins(network):
    """Get the original nodes with no predecessors"""
    origins = []
    for node in network.nodes:
        if len(network.predecessors(node)) == 0:
            origins.append(node)

    return origins

Introduction: We made a genetic probability predictor that uses punnet square math to predict a user's own or children's probability of having a certain disease or disorder. We use a family tree for the user to put in any information they have on phenotypes or genotypes of the disease they are testing for. We are using how certain diseases are genetically inherited to 

## Get DiGraph End

In [ ]:
def get_ends(network):
    """Get the ending nodes with no successors"""
    ends = []
    for node in network.nodes:
        if len(network.successors(node)) == 0:
            ends.append(node)

    return ends

# Class Definitions

## Person
possibly unused

In [ ]:
class Person():
    """A class to hold information on a person"""
    def __init__(self, phenotype, name, tree, genotype="", parents=[], children=[], **kwargs):
        self.phenotype = phenotype
        self.tree = tree
        self.genotype = genotype
        self.name = name
        self.parents = parents
        self.children = children
        self.extras = kwargs

    def set_phenotype(self, new_phenotype):
        self.phenotype = new_phenotype

    def set_genotype(self, new_genotype):
        self.genotype = new_genotype

    def get_name(self):
        return self.name

    def get_tree(self):
        return self.tree

    def get_genotype(self):
        return self.genotype

    def get_phenotype(self):
        return self.phenotype

    def get_children(self):
        return self.children

    def get_parents(self):
        return self.parents

## Family Subunit

In [ ]:
class FamilySubunit():
    def __init__(self, sub_network):
        pass

## Family Tree

In [ ]:
class FamilyTree():
    """A class to store the people in a family tree
       and the relationships between them. Also has
       methods to update the relationships."""
    def __init__(self, disease, people={}):
        self.graph = nx.DiGraph()
        self.disease = disease

        if len(people) != 0:
            for name, extras in people.items():
                pheno = extras['phenotype']
                geno = extras['genotype'] if 'genotype' in extras.keys() else None
                parents = extras['parents'] if 'parents' in extras.keys() else []
                children = extras['children'] if 'children' in extras.keys() else []
            
                self.add_person(pheno, geno, name, parents, children)

            self.update_probs()


    def add_person(self, phenotype, genotype=None, name=None, parents=[], children=[], **kwargs):
        """Adds a person to the family tree network graph"""

        # Makes the name of the person just a number if no name is provided
        if not name:
            # Determines what number person the new one is
            num = 0
            for node in self.graph.nodes:
                try:
                    val = int(node)
                except ValueError:
                    pass
                else:
                    num += 1

            name = str(num)

        if not genotype:
            genotype = disease.initial(phenotype)

        kwargs['phenotype'] = phenotype
        kwargs['genotype'] = genotype

        # Adds person to the graph and passes on any kwargs
        self.graph.add_nodes_from([(name, kwargs)])

        if len(parents)+len(children) > 0:
            # Add parents and children
            for parent in parents:
                self.graph.add_edge(parent, name)

            for child in children:
                self.graph.add_edge(name, child)
                
            self.update_probs()

    def add_relationship(self, parent, child):
        """Connects two people on the graph"""
        self.graph.add_edge(parent, child)
        self.update_probs()

    def remove_person(self, person):
        """Removes a person from the graph and updates
           the numbering of people in the graph"""
        self.graph.remove_node(person)

        try:
            val = int(person)
        except ValueError:
            pass
        else:
            # Counter to store how many unnamed people exist in the tree
            counter = 0
            # Loop through the nodes in the network to update
            for node in list(self.graph.nodes):
                try:
                    # Check if node was given a number for a name
                    val = int(node)
                except ValueError:
                    # If it was given an actual name move on
                    pass
                else:
                    new_name = str(counter)

                    # Grab the info attributed to the node
                    node_info = self.graph.nodes[node]

                    # Get the predecessors and successors of the node
                    sucs = self.graph.successors(node)
                    pres = self.graph.predecessors(node)

                    # Remove the old node and create a new one with an updated number
                    self.graph.remove_node(node)
                    self.graph.add_nodes_from([(new_name, node_info)])

                    # Re-add connections
                    for pre in pres:
                        self.graph.add_edge(pre, new_name)

                    for suc in sucs:
                        self.graph.add_edge(new_name, suc)

                    # Increment the counter
                    counter += 1

        self.update_probs()

    def get_person_info(self, person, information):
        """Get given information from a person in the graph"""
        output = {}
        for info in information:
            output[info] = self.graph.nodes[person][info]

        return output

    def save_tree(self, path):
        with open(path) as file:
            dump(self, file)

    def get_subunit(self, parent):
        """Gets the family unit of a person assuming they are a parent"""
        parents = [parent]
        children = list(self.graph[parent].keys())
        for child in children:
            for pred in self.graph.predecessors(child):
                if pred not in parents:
                    parents.append(pred)

        return nx.subgraph(self.graph, parents + children)

    def update_probs(self):
        pass

# Jon's Code

In [ ]:
def xlinked(father, mother, child):
    """" X is healthy, x is unhealthy, number is percent infected"""""
    if child == "M":
        if father == "XY" or father =="xY":
            if mother == "xx":
                return 1
            if mother == "Xx" or mother == "xX":
                return .5
            if mother == "XX":
                return 0
    else:
        if father == "xY":
            #all female children carriers
            if mother == "xx":
                return 1
            if mother == "XX":
                # child is carrier 1
                return 0
            if mother == "Xx" or mother == "xX":
                # child is carrier .75
                return .5
        else:
            if mother == "xx":
                # all children carriers
                return 0
            if mother == "Xx" or mother == "xX":
                # half of Daughter carriers
                return 0
            if mother == "XX":
                return 0

# Remi's Code

In [ ]:
def recessive(parent1, parent2):
    '''calculates likelihood of having disease that is autosomal recessive'''
    #give parent one's probability contribution
    if parent1 == "HH":
        num1 = 0
    elif parent1 == "Hh":
        num1 = 0.5
    elif parent1 == "hh":
        num1 = 1
    
    if parent2 == "HH":
        num2 = 0
    elif parent2== "Hh":
        num2 = 0.5
    elif parent2 == "hh":
        num2 = 1
        
    return (num1 * num2)

In [ ]:
def dominant(parent1, parent2):
    
    return 1 - recessive(parent1, parent2)

In [ ]:
def codominant(parent1, parent2):
    '''calculates likelihood of having disease that is autosomal codominant'''
    #give parent one's probability contribution
    if parent1 == "HH":
        numH = 1
        numh = 0
    elif parent1 == "Hh":
        numH = 0.5
        numh = 0.5
    elif parent1 == "hh":
        numH = 0
        numh = 1
    
    if parent2 == "HH":
        num2_H = 1
        num2_h = 0
    elif parent2== "Hh":
        num2_H = 0.5
        num2_h = 0.5
    elif parent2 == "hh":
        num2_H = 0
        num2_h = 1
        
    return (numH * num2_H, (numH * num2_h) + (numh * num2_H), numh * num2_h)

In [ ]:
codominant("Hh", "Hh")

## Base Inheritance Modes

## Specific Diseases

# Main Code